In [1]:
# prompt: conectar al drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#install librerias


In [ ]:
#!pip install rasterio
#!python /content/drive/MyDrive/UPCProyecto/utils/divide_mask.py

#2m
#Total mask tiles saved: 22967
#Total empty tiles: 39533

In [ ]:
#!python /content/drive/MyDrive/UPCProyecto/utils/divide_img.py


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm


In [26]:
import sys
import os

sys.path.append("/content/drive/MyDrive/UPCProyecto/") # Assuming 'utils' is in 'src'

from utils.dataset import SegmentationDataset, get_transforms # Now you should be able to import
from model import UNet
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt



In [27]:
num_classes = 12
batch_size = 3
epochs = 1
lr = 0.0001

# Dataset y DataLoader
train_images_dir = "/content/drive/MyDrive/UPCProyecto/Data/img368"
train_masks_dir = "/content/drive/MyDrive/UPCProyecto/Data/mask368"


In [6]:
# Create dataset and split into train and validation sets
dataset = SegmentationDataset(train_images_dir, train_masks_dir, transform=get_transforms())
# Set a fixed seed for reproducibility
seed = 42  # You can choose any seed value
torch.manual_seed(seed)
num_workers = 4 # Utilize multiple workers for data loading

In [7]:
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size   # 20% for validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [8]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True) # Added validation loader


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [10]:

# Normalización de pesos para la función de pérdida (ignorando clases 0)
class_weights = torch.ones(num_classes).to(device)
class_weights[0] = 0  # Clase "mar" ignorada

In [11]:



criterion = nn.CrossEntropyLoss(weight=class_weights, ignore_index=0)  # Ignoramos la clase 0

# Modelo y optimizador
model = UNet(num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Scheduler para reducir el learning rate si la pérdida no mejora
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 179MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [13]:
torch.cuda.device_count()

1

In [15]:
if torch.cuda.device_count() >= 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
else:
  print("hola")

Using 1 GPUs!


In [ ]:

# Función de entrenamiento
def train():
    best_loss = float('inf')  # Para rastrear la mejor pérdida

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        class_correct = torch.zeros(num_classes).to(device)
        class_total = torch.zeros(num_classes).to(device)

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)
        for images, masks in loop:
            images, masks = images.to(device), masks.to(device)

            # Convertir la clase 41 en 0 para que sea ignorada

            optimizer.zero_grad()
            outputs = model(images)

            # Calcular pérdida
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Calcular accuracy por clase (sin contar clases 0 y 41)
            _, predicted = torch.max(outputs, 1)
            for i in range(num_classes):
                if i not in [0]:  # Ignorar clases no deseadas
                    class_correct[i] += (predicted[masks == i] == i).sum().item()
                    class_total[i] += (masks == i).sum().item()

            # Actualizar barra de progreso
            loop.set_postfix(loss=loss.item())

        # Resultados de la época
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

        for i in range(num_classes):
            if i not in [0] and class_total[i] > 0:
                acc = 100 * class_correct[i] / class_total[i]
                print(f"Clase {i}: {acc:.2f}% accuracy")

        # Ajustar el learning rate si la loss no mejora
        scheduler.step(epoch_loss)

        # Guardar el mejor modelo
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            name = f"model_{epoch}_{epoch_loss:.4f}.pth"
            torch.save(model.state_dict(), name)
            print(f"Mejor modelo guardado con Loss: {epoch_loss:.4f}")

train()

In [1]:
from google.colab import drive
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import sys
import os

sys.path.append("/content/drive/MyDrive/UPCProyecto/")

In [2]:


from utils.dataset import SegmentationDataset, get_transforms
from model import UNet
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:





num_classes = 12
batch_size = 20 # Increased batch size
epochs = 100 # Increased epochs
lr = 0.001 # Increased learning rate
# Using AdamW optimizer, often more stable than Adam
# https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
optimizer_name = 'AdamW' #param ['Adam', 'AdamW', 'SGD']


# Dataset y DataLoader
train_images_dir = "/content/drive/MyDrive/UPCProyecto/Data/img368"
train_masks_dir = "/content/drive/MyDrive/UPCProyecto/Data/mask368"

# Create dataset and split into train and validation sets
dataset = SegmentationDataset(train_images_dir, train_masks_dir, transform=get_transforms())
seed = 42
torch.manual_seed(seed)
num_workers = os.cpu_count() # Use all available CPU cores for data loading
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

class_weights = torch.ones(num_classes).to(device)
class_weights[0] = 0

criterion = nn.CrossEntropyLoss(weight=class_weights, ignore_index=0)

# Modelo y optimizador
model = UNet(num_classes=num_classes).to(device)

if optimizer_name == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
elif optimizer_name == 'AdamW':
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # Use AdamW
elif optimizer_name == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9) # Use SGD with momentum


scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True) # More aggressive scheduler

if torch.cuda.device_count() >= 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

def train():
    best_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        class_correct = torch.zeros(num_classes).to(device)
        class_total = torch.zeros(num_classes).to(device)

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)
        for images, masks in loop:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            loop.set_postfix(loss=loss.item())

            # Calcular accuracy por clase (sin contar clases 0 y 41)
            _, predicted = torch.max(outputs, 1)
            for i in range(num_classes):
                if i not in [0]:  # Ignorar clases no deseadas
                    class_correct[i] += (predicted[masks == i] == i).sum().item()
                    class_total[i] += (masks == i).sum().item()

            # Actualizar barra de progreso
            loop.set_postfix(loss=loss.item())


        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")


        class_accuracies = {}
        for i in range(num_classes):
                    if i not in [0] and class_total[i] > 0:
                        acc = 100 * class_correct[i] / class_total[i]
                        class_accuracies[i] = acc
                        print(f"Clase {i}: {acc:.2f}% accuracy")
                    elif i not in [0] and class_total[i] == 0:
                      print(f"Clase {i}: No samples found for accuracy calculation")

# Calculate overall accuracy (considering only classes with samples)
        overall_accuracy = 0
        valid_classes_count = 0
        for i in class_accuracies:
          overall_accuracy += class_accuracies[i]
          valid_classes_count+=1

        if valid_classes_count > 0:
            overall_accuracy = overall_accuracy/valid_classes_count
            print(f"Overall accuracy (excluding class 0): {overall_accuracy:.2f}%")
        else:
            print("No valid classes found for overall accuracy calculation")



        scheduler.step(epoch_loss)

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            name = f"model_{epoch}_{epoch_loss:.4f}.pth"
            torch.save(model.state_dict(), name)
            print(f"Mejor modelo guardado con Loss: {epoch_loss:.4f}")

train()


Usando dispositivo: cuda


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 154MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Using 1 GPUs!


Epoch 1/100: 100%|██████████| 103/103 [05:00<00:00,  2.91s/it, loss=1.22]


Epoch 1/100, Loss: 1.4218
Clase 1: 0.00% accuracy
Clase 2: 96.25% accuracy
Clase 3: 0.20% accuracy
Clase 4: 0.73% accuracy
Clase 5: 0.00% accuracy
Clase 6: 0.00% accuracy
Clase 7: 2.55% accuracy
Clase 8: 0.00% accuracy
Clase 9: 0.01% accuracy
Clase 10: 0.00% accuracy
Clase 11: No samples found for accuracy calculation
Overall accuracy (excluding class 0): 9.97%
Mejor modelo guardado con Loss: 1.4218


...

Epoch 98/100, Loss: 0.5642
Clase 1: 85.06% accuracy
Clase 2: 90.49% accuracy
Clase 3: 66.58% accuracy
Clase 4: 34.51% accuracy
Clase 5: 87.53% accuracy
Clase 6: 49.51% accuracy
Clase 7: 25.90% accuracy
Clase 8: 59.62% accuracy
Clase 9: 69.28% accuracy
Clase 10: 41.64% accuracy
Clase 11: No samples found for accuracy calculation
Overall accuracy (excluding class 0): 61.01%
Mejor modelo guardado con Loss: 0.5642


Epoch 99/100: 100%|██████████| 103/103 [00:35<00:00,  2.87it/s, loss=0.601]


Epoch 99/100, Loss: 0.5601
Clase 1: 85.21% accuracy
Clase 2: 90.60% accuracy
Clase 3: 66.65% accuracy
Clase 4: 34.60% accuracy
Clase 5: 88.02% accuracy
Clase 6: 48.99% accuracy
Clase 7: 26.16% accuracy
Clase 8: 59.68% accuracy
Clase 9: 69.53% accuracy
Clase 10: 41.91% accuracy
Clase 11: No samples found for accuracy calculation
Overall accuracy (excluding class 0): 61.14%
Mejor modelo guardado con Loss: 0.5601


Epoch 100/100: 100%|██████████| 103/103 [00:35<00:00,  2.87it/s, loss=0.548]


Epoch 100/100, Loss: 0.5577
Clase 1: 85.27% accuracy
Clase 2: 90.63% accuracy
Clase 3: 66.70% accuracy
Clase 4: 34.66% accuracy
Clase 5: 87.82% accuracy
Clase 6: 49.12% accuracy
Clase 7: 27.14% accuracy
Clase 8: 59.59% accuracy
Clase 9: 70.23% accuracy
Clase 10: 41.94% accuracy
Clase 11: No samples found for accuracy calculation
Overall accuracy (excluding class 0): 61.31%
Mejor modelo guardado con Loss: 0.5577
